## Note: Handling overlapping forecast runs in evaluation

- In this notebook we often read one specific run via `storage_client.download_forecast_run(RUN_ID, spark)`.
- When multiple runs overlap on the same dates, mixing them can distort time-series evaluation.
- Use `load_latest_per_date_forecasts(storage_client, spark)` to read all runs as-is and, for each date, keep only the data from the maximum `run_id` that contains that date.

Example:
```python
from functions_library.forecast_evaluation import load_latest_per_date_forecasts

latest_per_date = load_latest_per_date_forecasts(storage_client, spark)
latest_per_date.show(20)
```

Selection rule:
- For each distinct `date` across all runs, retain only rows from the highest `run_id` available on that date (all products for that date are kept from that run).


# Forecast Performance Scorecard

This scorecard provides a systematic, top-down approach to analyze forecast performance and identify areas for model improvement. The process follows a drill-down methodology from high-level business metrics to individual product analysis.

## Scorecard Process Flow:

### 1. **Portfolio Overview** 
Start with the big picture - how is the entire business performing against forecasts? This gives you the overall health of your forecasting system and identifies if there are systematic issues affecting the whole portfolio.

### 2. **Category Performance Analysis**
Drill down into sales categories (Very Low, Low, Alive, Medium, Winning, High Winning) to identify which product segments are driving forecast errors. This helps prioritize which categories need immediate attention.

### 3. **Error Distribution Deep Dive**
For problematic categories, analyze the distribution of forecast errors to understand:
- Are we consistently over-forecasting or under-forecasting?
- What's the magnitude of errors?
- Which products are the biggest contributors to poor performance?

### 4. **Individual Product Investigation**
Focus on the worst-performing products to understand root causes:
- Are there data quality issues?
- Is the product experiencing unusual patterns (promotions, stockouts, seasonality)?
- Are there external factors affecting demand?

### 5. **Actionable Insights Generation**
Based on the analysis, identify specific actions:
- Which products need model retraining?
- What data quality issues need fixing?
- Which categories need different forecasting approaches?
- What external factors should be incorporated into the model?

## How to Use This Scorecard:

1. **Run the complete notebook** to get the full analysis
2. **Start with the business-level view** to understand overall performance
3. **Identify problematic categories** using the error distribution plots
4. **Drill down into specific products** that are driving poor performance
5. **Document findings** walk through lots of skus and identofy dominnat issues then correct. Don't focus on correcting specific edge cases. This will end up messing working cases



# Scorecard

In [1]:
# =============================================================================
# ALL IMPORTS - CONSOLIDATFEVAL IN FIRST CELL
# =============================================================================

# Standard library imports
import sys
import importlib
from importlib import reload

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# PySpark imports
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window

# Local imports
import configuration as CONFIG
from functions_library.supabase_connection import SupabaseClient
import functions_library.forecast_evaluation as FEVAL

# =============================================================================
# MODULE RELOADING - Ensure latest versions are loaded
# =============================================================================
importlib.reload(CONFIG)
importlib.reload(sys.modules['functions_library.supabase_connection'])
importlib.reload(sys.modules['functions_library.forecast_evaluation'])

# =============================================================================
# SPARK INITIALIZATION
# =============================================================================
spark = SparkSession.builder.appName('ForecastEvaluation').master('local[1]').getOrCreate()
spark.sparkContext.setLogLevel("WARN")

# =============================================================================
# SUPABASE CLIENT INITIALIZATION
# =============================================================================
storage_client = SupabaseClient()

In [2]:
EVALUATION_START_DATE = "2025-08-01"
EVALUATION_END_DATE = "2025-08-07"

In [3]:
products = storage_client.batch_read_table("products", spark)
prod_day_history = storage_client.download_processed_features(spark)
prod_day_forecast = FEVAL.load_latest_per_date_forecasts(storage_client, spark)


In [4]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [5]:
prod_day_forecast = FEVAL.load_latest_per_date_forecasts(storage_client, spark)


In [6]:
prod_day_history.count()

185931

In [7]:
prod_day_history.select(F.max("date")).show()

+----------+
| max(date)|
+----------+
|2025-10-14|
+----------+



In [8]:
prod_day_forecast.show()

+----------+--------------------+--------------------+------------+--------------+------+---------------+--------------------+------+
|      date|          product_id|  age_sales_category|age_category|sales_category| style|forecast_method|            forecast|run_id|
+----------+--------------------+--------------------+------------+--------------+------+---------------+--------------------+------+
|2025-08-05|4e929c59-7ce9-4a6...|04| Mature_04| Ma...|  04| Mature|  02| Very Low| MWT99|        croston|  0.6143396226415094|     1|
|2025-08-05|6ebebae4-4e30-4a0...|04| Mature_04| Ma...|  04| Mature|  02| Very Low|MW1494|        croston|  0.4772881355932204|     1|
|2025-08-05|b4f592fd-c677-41f...|04| Mature_04| Ma...|  04| Mature|     04| Alive|MW1331|        prophet|   3.473988864823687|     1|
|2025-08-05|c6c76528-116b-43e...|04| Mature_04| Ma...|  04| Mature|  02| Very Low|MW1304|        croston|  0.3360000000000001|     1|
|2025-08-05|ec57f9d3-6dc5-464...|04| Mature_04| Ma...|  04| Ma

In [9]:
evaluation_data = (prod_day_forecast
                   .join(prod_day_history.select("product_id", "date", "sales_units"), on=["product_id", "date"], how="inner")
                   .groupBy("product_id", "style", "age_sales_category", "age_category", "sales_category", "forecast_method")
                   .agg(F.sum("sales_units").alias("sales_units"),
                        F.sum("forecast").alias("forecast")))
evaluation_data = FEVAL.calculate_bias_and_error_bins(evaluation_data)

In [10]:
validation_data = (prod_day_history
                   .join(prod_day_forecast.select("product_id", "date", "forecast_method", "forecast"), on=["product_id", "date"],how="left"))
validation_data = FEVAL.calculate_bias_and_error_bins(validation_data)

# Business Summary

In [11]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [12]:
pdf, plot = FEVAL.plot_business_level(validation_data, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()

Evaluation frame (2025-08-01 - 2025-08-07):
  Total Sales: 3402
  Total Forecast: 2094.5255144230105
  Over/Under Forecast: -1307.4744855769895 units (-38.4%)


# Overall Evaluation

In [13]:
# Define per-bin good thresholds (understock / overstock)
GOOD_THRESHOLD = {
    "02| Very Low": (5, 10),
    "03| Low": (5, 10),
    "04| Alive": (5, 20),
    "05| Medium": (5, 20),
    "06| Winning": (5, 20), # Bias
    "07| High Winning": (5, 30) # Bias
}

In [ ]:
df, plot = FEVAL.plot_attribute_distribution(evaluation_data, ["sales_category"])
plot.show()


In [15]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [16]:
FEVAL.plot_accuracy_distribution(evaluation_data, method="bias").show()
FEVAL.plot_accuracy_distribution(evaluation_data, method="error").show()

In [17]:
df = (evaluation_data
    .where(~(F.col("sales_category").isin(["02| Very Low", "03| Low", "07| High Winning"]))))
FEVAL.plot_accuracy_distribution(df, method="bias").show()

# Investigate Sales Categories

In [18]:
SELECTFEVAL_SALES_BIN = "07| High Winning" 
METHOD = "bias"

In [19]:
evaluation_data_category = evaluation_data.where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
FEVAL.analyze_category_performance(evaluation_data_category)

,product_count,sales_units,forecast,error
0,11,2322,3144.063109,822.063109


In [20]:
FEVAL.plot_accuracy_distribution(evaluation_data_category, method=METHOD).show()

# Investigate Products

In [21]:
SELECTFEVAL_BIAS_BIN = "22| > +50"
LIMIT = 10

In [22]:
(evaluation_data_category
    .where(F.col("sales_category") == SELECTFEVAL_SALES_BIN)
    .where(F.col("bias_bin").rlike(SELECTFEVAL_BIAS_BIN))
    .orderBy(F.desc("error")).limit(LIMIT).toPandas())

,product_id,style,age_sales_category,age_category,sales_category,forecast_method,sales_units,forecast,bias,error,bias_bin,error_bin
0,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,MW1537,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,727,1650.932848,1.270884,923.932848,22| > +100,22| > +50
1,1392c7f9-172f-4069-9b61-f9504c7ee281,MW1545,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,prophet,50,115.622111,1.312442,65.622111,22| > +100,22| > +50


In [23]:
reload(FEVAL)

<module 'functions_library.forecast_evaluation' from 'c:\\Users\\Cognira\\Desktop\\Aya - Forecast\\functions_library\\forecast_evaluation.py'>

In [25]:
# Example usage
SELECTED_STYLE = "MW1537" 
pdf_product, plot = FEVAL.plot_product_details_daily_short(validation_data, SELECTED_STYLE, products_df=products, EVALUATION_START_DATE=EVALUATION_START_DATE, EVALUATION_END_DATE=EVALUATION_END_DATE)
plot.show()


Product Title: MW1537 - عباية صيفية ملونة
Price: 252.17
Status: ACTIVE


In [27]:
prod_day_forecast.where(F.col("style") == SELECTED_STYLE).orderBy("date").toPandas()

,date,product_id,age_sales_category,age_category,sales_category,style,forecast_method,forecast,run_id
0,2025-08-02,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
1,2025-08-03,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
2,2025-08-04,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
3,2025-08-05,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
4,2025-08-06,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
5,2025-08-07,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
6,2025-08-08,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,croston,16.272542,1
7,2025-08-09,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,prophet,90.308436,2
8,2025-08-10,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,prophet,79.304845,2
9,2025-08-11,3a1bc094-73ba-47e6-a039-2cd80dd83f4f,04| Mature_04| Mature_07| High Winning,04| Mature,07| High Winning,MW1537,prophet,80.900597,2
